##IMPORTS


In [ ]:
!pip install rdkit
!pip install scikit-learn
!git clone https://github.com/HESTHER/espsim.git
%cd espsim
!pip install -e .
from espsim import GetEspSim, EmbedAlignScore
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors, Lipinski, Crippen
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import MACCSkeys
from rdkit.DataStructs import TanimotoSimilarity
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, AllChem
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from google.colab import drive
import os
import csv
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.0 MB/s eta 0:00:00
Cloning into 'espsim'...
remote: Enumerating objects: 6651, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 6651 (delta 94), reused 178 (delta 91), pack-reused 6464
Receiving objects: 100% (6651/6651), 45.17 MiB | 28.00 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/espsim
Obtaining file:///content/espsim
  Preparing metadata (setup.py) ... done
  Running setup.py develop for espsim


##Defined FUNCTIONS

In [ ]:
# Function to calculate molecular shape descriptors
def calculate_molecular_shape_descriptors(mol):
    try:
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(mol)
        surface_area = Descriptors.TPSA(mol)  # Placeholder for surface area
        return {'SurfaceArea': surface_area}
    except ValueError as e:
        print(f"Error calculating molecular shape descriptors: {e}")
# Function to calculate charge distribution descriptors using Gasteiger charges
def calculate_charge_distribution(mol):
    AllChem.ComputeGasteigerCharges(mol)
    partial_charges = [atom.GetDoubleProp('_GasteigerCharge') for atom in mol.GetAtoms()]
    return {'PartialCharges': partial_charges}

# Calculate Shape and ESP similarity using EmbedAlignScore
def calculate_espsim_descriptors(mol1, mol2):
  try:

    sim_shape, sim_esp = EmbedAlignScore(mol1, mol2, getBestESP=True)

    return sim_shape, sim_esp
  except ValueError as e:
        print(f"Error calculating ESP similarity: {e}")
        return None, None

def tanimoto(reference_molecule, researched_molecule):
    """
    Calculates Tanimoto similarity between two molecules using Morgan fingerprints.

    Parameters:
    - reference_molecule: RDKit Mol object of the reference molecule
    - researched_molecule: RDKit Mol object of the researched molecule

    Returns:
    - similarity_score: Tanimoto similarity score between the molecules' fingerprints
    """
    fp = AllChem.GetMorganFingerprintAsBitVect(reference_molecule, 2, nBits=1024)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(researched_molecule, 2, nBits=1024)

    same_parts = set(fp.GetOnBits()) & set(fp2.GetOnBits())
    different_parts = set(fp.GetOnBits()) | set(fp2.GetOnBits())

    similarity_score = len(same_parts) / len(different_parts)
    return similarity_score

def normalize_similarity(reference_desc, smiles_desc, similarity_score):
    """
    Normalizes similarity score based on descriptor values.

    Parameters:
    - reference_desc: Dictionary of descriptor values for the reference molecule
    - smiles_desc: Dictionary of descriptor values for the researched molecule
    - similarity_score: Similarity score between the molecules

    Returns:
    - normalized_score: Normalized similarity score
    """
    desc_values = []
    for desc, value in smiles_desc.items():
        if isinstance(value, list):
            ref_partial_charges = reference_desc[desc]
            if len(ref_partial_charges) != len(value):
                continue  # Skip if lengths of lists are different
            partial_charges_similarity = sum(abs(ref_partial_charges[i] - value[i]) for i in range(len(ref_partial_charges)))
            desc_values.append(partial_charges_similarity)
        else:
            desc_values.append(abs(reference_desc[desc] - value))

    distance = np.linalg.norm(desc_values)
    normalized_score = similarity_score / (1 + distance)
    return normalized_score

def calculate_overall_similarity_with_normalization(ref_descriptors, gen_descriptors, ref_partial_charges, reference_molecule, researched_molecule):
    """
    Calculates overall similarity with normalization based on descriptors.

    Parameters:
    - ref_descriptors: Dictionary of descriptors for the reference molecule
    - gen_descriptors: Dictionary of descriptors for the researched molecule
    - ref_partial_charges: Reference partial charges

    Returns:
    - normalized_similarity: Normalized overall similarity score
    """
    overall_similarity = 0
    common_keys = set(ref_descriptors.keys()).intersection(gen_descriptors.keys())

    for desc in common_keys:
        if desc == 'PartialCharges':
            ref_partial_charges = ref_descriptors[desc]
            gen_partial_charges = gen_descriptors[desc]
            if len(ref_partial_charges) != len(gen_partial_charges):
                continue  # Skip if lengths of lists are different
            partial_charges_similarity = sum(abs(ref_partial_charges[i] - gen_partial_charges[i]) for i in range(len(ref_partial_charges)))
            overall_similarity += partial_charges_similarity

    similarity_score = tanimoto(reference_molecule, researched_molecule)
    normalized_similarity = normalize_similarity(ref_descriptors, gen_descriptors, overall_similarity + similarity_score)
    return normalized_similarity


##Working with the reference descriptors

In [ ]:
drive.mount('/content/drive')
results_folder = '/content/drive/My Drive/results'  # Replace this with the path to your 'results' folder
os.chdir(results_folder)

Mounted at /content/drive


In [ ]:
reference_smiles = "CNc1ncc2cc(-c3ccc(-c4ncccc4F)cc3Cl)c(=O)n(C[C@H]3OC[C@H](N)CO3)c2n1"

reference_mol = Chem.MolFromSmiles(reference_smiles)
reference_mol = Chem.AddHs(reference_mol)
Chem.SanitizeMol(reference_mol)
AllChem.ComputeGasteigerCharges(reference_mol)


ref_descriptors = {
    'MolecularWeight': Descriptors.MolWt(reference_mol),
    'NumHDonors': Lipinski.NumHDonors(reference_mol),
    'NumHAcceptors': Lipinski.NumHAcceptors(reference_mol),
    'TPSA': Descriptors.TPSA(reference_mol),
    'MolLogP': Descriptors.MolLogP(reference_mol),
    'NumRotatableBonds': Lipinski.NumRotatableBonds(reference_mol),
    'NumRings': reference_mol.GetRingInfo().NumRings(),
    'ClogP': Crippen.MolLogP(reference_mol),
    'MolarRefractivity': Crippen.MolMR(reference_mol)
}

charge_distribution = calculate_charge_distribution(reference_mol)
molecular_shape = calculate_molecular_shape_descriptors(reference_mol)
# Add additional descriptors to the reference descriptors

ref_descriptors.update(charge_distribution)
ref_descriptors.update(molecular_shape)





<h1> GENERATED SMILES<h1/>

In [ ]:
# print("Reference Molecule Descriptors:")
# for desc, score in ref_descriptors.items():
#     print(f"{desc}: {score}")
# print("\n")

# # Open the generated smiles file and initialize the reference
# output_data = []
# for folder in os.listdir('.'):
#     folder_path = os.path.join(results_folder, folder)
#     smiles_file = os.path.join(folder_path, './final_ranking/final_smiles.txt')
#     print(folder)
#     if os.path.exists(smiles_file):
#         with open(smiles_file, 'r') as file:
#             generated_smiles = [line.strip() for line in file.readlines()]

#             for i, smile in enumerate(generated_smiles):
#                 gen_mol = Chem.MolFromSmiles(smile.strip())

#                 if gen_mol is None:
#                     print(f"Invalid SMILES for generated molecule {i+1} in folder {folder}")
#                     continue

#                 gen_mol = Chem.AddHs(gen_mol)
#                 Chem.SanitizeMol(gen_mol)

#                 gen_descriptors = {
#                     'MolecularWeight': Descriptors.MolWt(gen_mol),
#                     'NumHDonors': Lipinski.NumHDonors(gen_mol),
#                     'NumHAcceptors': Lipinski.NumHAcceptors(gen_mol),
#                     'TPSA': Descriptors.TPSA(gen_mol),
#                     'MolLogP': Descriptors.MolLogP(gen_mol),
#                     'NumRotatableBonds': Lipinski.NumRotatableBonds(gen_mol),
#                     'NumRings': gen_mol.GetRingInfo().NumRings(),
#                     'ClogP': Crippen.MolLogP(gen_mol),
#                     'MolarRefractivity': Crippen.MolMR(gen_mol)
#                 }

#                 # Calculate additional descriptors
#                 print(folder)
#                 molecular_shape = calculate_molecular_shape_descriptors(gen_mol)
#                 charge_distribution = calculate_charge_distribution(gen_mol)

#                 # Add additional descriptors to the generated descriptors
#                 if molecular_shape is not None and charge_distribution is not None:

#                     gen_descriptors.update(molecular_shape)
#                     gen_descriptors.update(charge_distribution)

#                     similarity = tanimoto(reference_mol, gen_mol)

#                     # Calculate ESpsim descriptors
#                     espsim_descriptors = calculate_espsim_descriptors(gen_mol, reference_mol)

#                     if espsim_descriptors[0] is not None and espsim_descriptors[1] is not None:
#                         # Process and print results

#                         overall_similarity = calculate_overall_similarity_with_normalization(ref_descriptors, gen_descriptors, ref_descriptors['PartialCharges'], reference_mol, gen_mol)

#                         molecule_data = {
#                             'GeneratedSMILES': smile,
#                             'SimilarityToReference': similarity,
#                             'ESP': espsim_descriptors[1][0],
#                             'shape': espsim_descriptors[0][0],
#                             'MolecularWeight': gen_descriptors['MolecularWeight'],
#                             'NumHDonors': gen_descriptors['NumHDonors'],
#                             'NumHAcceptors': gen_descriptors['NumHAcceptors'],
#                             'TPSA': gen_descriptors['TPSA'],
#                             'MolLogP': gen_descriptors['MolLogP'],
#                             'NumRotatableBonds': gen_descriptors['NumRotatableBonds'],
#                             'NumRings': gen_descriptors['NumRings'],
#                             'ClogP': gen_descriptors['ClogP'],
#                             'MolarRefractivity': gen_descriptors['MolarRefractivity'],
#                             'OverallSimilarityToReference': overall_similarity,
#                             'Folder': folder,
#                         }

#                         output_data.append(molecule_data)
#                     else:
#                         print(f"Error calculating ESpsim descriptors for generated molecule {i + 1} in folder {folder}")
#                 else:
#                     print(f"Error calculating descriptors for generated molecule {i + 1} in folder {folder}")

# # Define the CSV file name
# csv_file = 'output_data.csv'

# # Define the header for the CSV file based on the keys in the dictionaries
# header = output_data[0].keys() if output_data else []

# # Write data to CSV file
# with open(csv_file, 'w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=header)
#     writer.writeheader()
#     writer.writerows(output_data)

# print(f"Output data saved to {csv_file}")


Reference Molecule Descriptors:
MolecularWeight: 496.92999999999984
NumHDonors: 2
NumHAcceptors: 9
TPSA: 117.18
MolLogP: 3.0549
NumRotatableBonds: 7
NumRings: 5
ClogP: 3.0549
MolarRefractivity: 130.6471
PartialCharges: [0.006240181966910488, -0.35723618893946074, 0.2238909285176417, -0.22090316127392848, 0.04183340626773754, 0.03689617347043797, -0.03620520705938575, 0.049455391936802665, 0.005891493178862181, -0.05200467410850485, -0.051981319491895155, 0.004635794689571188, 0.1058848782135312, -0.25317074860170913, 0.027717845460897242, -0.04085687250802989, -0.022508446655835592, 0.1489663061043503, -0.20459316059918922, -0.033336553719480945, 0.04926791071056592, -0.08363659546232967, 0.25981786926929623, -0.2686187347790643, -0.286860815588909, 0.07881946919000753, 0.17512038754994919, -0.34904597930462394, 0.0672291690036522, 0.051675258116658136, -0.3238070490622402, 0.0672291690036522, -0.34904597930462394, 0.148100004051332, -0.1946750914931198, 0.045452915801258924, 0.0454529

In [ ]:

print("Reference Molecule Descriptors:")
for desc, score in ref_descriptors.items():
    print(f"{desc}: {score}")
print("\n")

output_data = []

# Load SMILES from the CSV file
for file in os.listdir('.'):
  print(file)
  csv_file_path = os.path.join(results_folder, file)
  df = pd.read_csv(csv_file_path)



# Loop through each SMILES in the CSV file
for i, row in df.iterrows():
    smile = row['SMILES']
    gen_mol = Chem.MolFromSmiles(smile)

    if gen_mol is None:
        print(f"Invalid SMILES for generated molecule {i+1}")
        continue


    gen_mol = Chem.AddHs(gen_mol)
    Chem.SanitizeMol(gen_mol)

    gen_descriptors = {
        'MolecularWeight': Descriptors.MolWt(gen_mol),
        'NumHDonors': Lipinski.NumHDonors(gen_mol),
        'NumHAcceptors': Lipinski.NumHAcceptors(gen_mol),
        'TPSA': Descriptors.TPSA(gen_mol),
        'MolLogP': Descriptors.MolLogP(gen_mol),
        'NumRotatableBonds': Lipinski.NumRotatableBonds(gen_mol),
        'NumRings': gen_mol.GetRingInfo().NumRings(),
        'ClogP': Crippen.MolLogP(gen_mol),
        'MolarRefractivity': Crippen.MolMR(gen_mol)
    }

    # Calculate additional descriptors
    molecular_shape = calculate_molecular_shape_descriptors(gen_mol)
    charge_distribution = calculate_charge_distribution(gen_mol)

    # Add additional descriptors to the generated descriptors
    if molecular_shape is not None and charge_distribution is not None:

        gen_descriptors.update(molecular_shape)
        gen_descriptors.update(charge_distribution)

        similarity = tanimoto(reference_mol, gen_mol)

        # Calculate ESpsim descriptors
        espsim_descriptors = calculate_espsim_descriptors(gen_mol, reference_mol)


        if espsim_descriptors[0] is not None and espsim_descriptors[1] is not None:
            # Process and print results

            overall_similarity = calculate_overall_similarity_with_normalization(ref_descriptors, gen_descriptors, ref_descriptors['PartialCharges'], reference_mol, gen_mol)

            molecule_data = {
                'GeneratedSMILES': smile,
                'SimilarityToReference': similarity,
                'ESP': espsim_descriptors[1][0],
                'shape': espsim_descriptors[0][0],
                'MolecularWeight': gen_descriptors['MolecularWeight'],
                'NumHDonors': gen_descriptors['NumHDonors'],
                'NumHAcceptors': gen_descriptors['NumHAcceptors'],
                'TPSA': gen_descriptors['TPSA'],
                'MolLogP': gen_descriptors['MolLogP'],
                'NumRotatableBonds': gen_descriptors['NumRotatableBonds'],
                'NumRings': gen_descriptors['NumRings'],
                'ClogP': gen_descriptors['ClogP'],
                'MolarRefractivity': gen_descriptors['MolarRefractivity'],
                'OverallSimilarityToReference': overall_similarity,
            }

            output_data.append(molecule_data)
        else:
            print(f"Error calculating ESpsim descriptors for generated molecule {i + 1}")
    else:
        print(f"Error calculating descriptors for generated molecule {i + 1}")

# else:
#     print(f"Error calculating ESpsim descriptors for generated molecule {i + 1}")

#         # gen_descriptors.update(charge_distribution)

#         # similarity = tanimoto(reference_mol, gen_mol)

#         print(f"Generated SMILES {i + 1}: {smile}")
#         print(f"Similarity to SIK2/SIK3 inhibitors: {similarity}")

#         espsim_descriptors = calculate_espsim_descriptors(gen_mol, reference_mol)

#         print(f"ESpsim descriptors for Generated SMILES {i + 1}: SHAPE =  {espsim_descriptors[0][0]}, ESP = {espsim_descriptors[0][0]}")
#         print("Similarity scores for individual descriptors:")
#         for desc, score in gen_descriptors.items():
#             print(f"{desc}: {score}")
#         print("\n")

#         overall_similarity = calculate_overall_similarity_with_normalization(ref_descriptors, gen_descriptors, ref_descriptors['PartialCharges'], reference_mol, gen_mol)
#         print(f"Overall similarity to reference molecule for SMILES {i + 1}: {overall_similarity}\n")

#         molecule_data = {
#             'GeneratedSMILES': smile,
#             'SimilarityToReference': similarity,
#             'ESP': espsim_descriptors[0][0],
#             'shape': espsim_descriptors[0][0],
#             'MolecularWeight': gen_descriptors['MolecularWeight'],
#             'NumHDonors': gen_descriptors['NumHDonors'],
#             'NumHAcceptors': gen_descriptors['NumHAcceptors'],
#             'TPSA': gen_descriptors['TPSA'],
#             'MolLogP': gen_descriptors['MolLogP'],
#             'NumRotatableBonds': gen_descriptors['NumRotatableBonds'],
#             'NumRings': gen_descriptors['NumRings'],
#             'ClogP': gen_descriptors['ClogP'],
#             'MolarRefractivity': gen_descriptors['MolarRefractivity'],
#             'OverallSimilarityToReference': overall_similarity,
#             'Folder': folder,
#         }

#         output_data.append(molecule_data)

# Define the CSV file name
csv_file = 'output_data.csv'

# Define the header for the CSV file based on the keys in the dictionaries
header = output_data[0].keys() if output_data else []

# Write data to CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(output_data)

print(f"Output data saved to {csv_file}")

In [ ]:
# # Write data to CSV file
# with open(csv_file, 'w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=header)
#     writer.writeheader()
#     writer.writerows(output_data)

# print(f"Output data saved to {csv_file}")